In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd '/content/drive/MyDrive/Semester5/PemrogramanSistemCerdas1/Prak_IvanderJoshSantoso/Sentimen_analiysis_ishihara'

/content/drive/MyDrive/Semester5/PemrogramanSistemCerdas1/Prak_IvanderJoshSantoso/Sentimen_analiysis_ishihara


In [5]:
!pip install google-play-scraper

In [6]:
from google_play_scraper import app
import pandas as pd
import numpy as np

!pip install Sastrawi

import nltk
import pandas as pd
import re
import time
import multiprocessing
import io
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import KFold # import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
from nltk.tokenize import TweetTokenizer
from collections import defaultdict
from datetime import timedelta
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.utils.multiclass import unique_labels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 1.7 MB/s eta 0:00:00


# **Scraping**

##Test Buta Warna - Ishihara

In [7]:
from google_play_scraper import Sort, reviews

#fungsi untuk memanggil riview
result, continuation_token = reviews(
    'com.divinememorygames.ishihara.color.blindness.test',
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
    count=1000, #defaultnya 100
)

In [8]:
# Dataframe dengan nama
ishihara = pd.DataFrame(np.array(result),columns=['review']) #membuat pandas DataFrame bernama 'ishihara'
ishihara = ishihara.join(pd.DataFrame(ishihara.pop('review').tolist()))
ishihara.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,477984dd-b679-443f-8b48-1d080951e42f,fatin nabila,https://play-lh.googleusercontent.com/a-/ALV-U...,Pada tes 'color plates test' kenapa saya selal...,4,1,2.33,2023-10-05 10:28:46,None,NaT,2.33
1,417a9719-f74e-455b-bf0d-07c2598d6501,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,ini bagus buat memperbaiki mata.tapi kalo bisa...,4,102,2.17,2020-03-25 14:58:53,None,NaT,2.17
2,4379cfaf-4d6e-46d6-8754-700ab1ed95d4,DakerXRz,https://play-lh.googleusercontent.com/a-/ALV-U...,"Saya sih gak buta warna, tapi makin kedepan lv...",1,20,None,2022-11-28 00:04:03,None,NaT,None
3,0b1a3d2e-47ac-44ab-898a-90b01a0f7d23,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Coba mengerjakan Ishihara Test sambil mengguna...,4,33,1.45,2018-12-11 23:13:24,"Thanks, we will fix it",2018-12-12 03:30:23,1.45
4,d20d69fc-5201-426c-be83-4df9a4736290,Azzahra Nova,https://play-lh.googleusercontent.com/a-/ALV-U...,Tolong aturin pake bahasa Indonesia,3,7,2.33,2023-08-22 09:05:38,None,NaT,2.33


In [9]:
ishihara[['content', 'score']].head()

,content,score
0,Pada tes 'color plates test' kenapa saya selal...,4
1,ini bagus buat memperbaiki mata.tapi kalo bisa...,4
2,"Saya sih gak buta warna, tapi makin kedepan lv...",1
3,Coba mengerjakan Ishihara Test sambil mengguna...,4
4,Tolong aturin pake bahasa Indonesia,3


In [10]:
def labelling(score):
    if score < 3:
        return 'Negatif'
    elif score == 3:
        return 'Netral'
    elif score > 3:
        return 'Positif'

ishihara['label'] = ishihara['score'].apply(labelling)
ishihara.head(200)
ishihara = ishihara[['content', 'score']]
ishihara.to_csv('rating1.csv', index=False)

##ColorPick Eyedropper

In [11]:
from google_play_scraper import Sort, reviews

#fungsi untuk memanggil riview
result, continuation_token = reviews(
    'com.hieuit.colorblindtest',
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
    count=1000, #defaultnya 100
)

In [12]:
# Dataframe dengan nama
ishihara = pd.DataFrame(np.array(result),columns=['review'])
ishihara = ishihara.join(pd.DataFrame(ishihara.pop('review').tolist()))
ishihara.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,80e49c29-8cd5-49e4-a5bf-a451c2cbc3ae,Hack Penyadap (Haier),https://play-lh.googleusercontent.com/a/ACg8oc...,"Sangat berguna untuk test buta warna, test pen...",5,2,None,2022-03-27 18:11:38,None,None,None
1,c4f8a4db-b661-4ba3-9731-90291b285b47,JEFFRY ARIANTO,https://play-lh.googleusercontent.com/a/ACg8oc...,mantap nih tapi tambahin lagi jangan sampe 24 aja,4,0,1.4,2022-06-23 23:19:49,None,None,1.4
2,70896840-520b-4c47-a2d3-7991bffa0d56,Bondjol Cool,https://play-lh.googleusercontent.com/a/ACg8oc...,"Bagus, tapi kok sampek 24 saja... tolong dong ...",4,10,1.4,2020-11-04 02:29:35,None,None,1.4
3,e7e60840-e5b5-4e08-a8f9-140469d66142,Iwan Nirwan,https://play-lh.googleusercontent.com/a/ACg8oc...,Boleh lahh,3,0,1.4,2022-06-24 23:12:42,None,None,1.4
4,f133523d-22ec-4bdd-8e10-1ddcfb03f596,Muhammad Fachrul Razi,https://play-lh.googleusercontent.com/a-/ALV-U...,Banyak iklan,1,0,1.4,2022-06-24 09:17:29,None,None,1.4


In [13]:
ishihara[['content', 'score']].head()

,content,score
0,"Sangat berguna untuk test buta warna, test pen...",5
1,mantap nih tapi tambahin lagi jangan sampe 24 aja,4
2,"Bagus, tapi kok sampek 24 saja... tolong dong ...",4
3,Boleh lahh,3
4,Banyak iklan,1


In [14]:
def labelling(score):
    if score < 3:
        return 'Negatif'
    elif score == 3:
        return 'Netral'
    elif score > 3:
        return 'Positif'

ishihara['label'] = ishihara['score'].apply(labelling)
ishihara.head(200)
ishihara = ishihara[['content', 'score']]
ishihara.to_csv('rating2.csv', index=False)

##Color Detector

In [15]:
from google_play_scraper import Sort, reviews

#fungsi untuk memanggil riview
result, continuation_token = reviews(
    'com.keesadens.colordetector',
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
    count=1000, #defaultnya 100
)

In [16]:
# Dataframe dengan nama
ishihara = pd.DataFrame(np.array(result),columns=['review'])
ishihara = ishihara.join(pd.DataFrame(ishihara.pop('review').tolist()))
ishihara.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,60c7758f-702a-4958-a062-7be0248fc228,Danial Haidar,https://play-lh.googleusercontent.com/a-/ALV-U...,"josss,.sangat mantap,.khususnya bagi yg bekerj...",5,7,3.1.1,2020-07-12 08:00:04,None,None,3.1.1
1,9fda2e86-3687-4d36-bce5-662bd1d4e71b,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Aplikasi inilah yang dari dulu saya cari cari ...,5,18,3.0.9,2018-11-19 13:16:09,None,None,3.0.9
2,119f4d1e-7703-4c31-99e3-1162f3f0aa8d,Reza Uzumaki Walkthrough Game,https://play-lh.googleusercontent.com/a/ACg8oc...,Sangat membantu buat yang mengalami buta warna,5,5,3.1.1,2022-05-03 02:43:52,None,None,3.1.1
3,057c4830-5c1c-49ba-9c03-c8fc70033c87,Ekobudi Santoso,https://play-lh.googleusercontent.com/a-/ALV-U...,Sangat membantu bagi kami yang buta warna,5,7,3.1.1,2021-03-24 14:57:42,None,None,3.1.1
4,c8df3bad-cae8-4785-b6d8-e2bc0627e034,Faruqo Faris234,https://play-lh.googleusercontent.com/a-/ALV-U...,sangat membantu bagi yang buta warna,5,0,None,2022-06-26 04:47:48,None,None,None


In [17]:
ishihara[['content', 'score']].head()

,content,score
0,"josss,.sangat mantap,.khususnya bagi yg bekerj...",5
1,Aplikasi inilah yang dari dulu saya cari cari ...,5
2,Sangat membantu buat yang mengalami buta warna,5
3,Sangat membantu bagi kami yang buta warna,5
4,sangat membantu bagi yang buta warna,5


In [18]:
def labelling(score):
    if score < 3:
        return 'Negatif'
    elif score == 3:
        return 'Netral'
    elif score > 3:
        return 'Positif'

ishihara['label'] = ishihara['score'].apply(labelling)
ishihara.head(200)
ishihara = ishihara[['content', 'score']]
ishihara.to_csv('rating3.csv', index=False)

##Color Grab

In [19]:
from google_play_scraper import Sort, reviews

#fungsi untuk memanggil riview
result, continuation_token = reviews(
    'com.loomatix.colorgrab',
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
    count=1000, #defaultnya 100
)

In [20]:
# Dataframe dengan nama
ishihara = pd.DataFrame(np.array(result),columns=['review'])
ishihara = ishihara.join(pd.DataFrame(ishihara.pop('review').tolist()))
ishihara.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,d83651f6-2492-4625-aa88-747a2331a313,Lloyd Dream,https://play-lh.googleusercontent.com/a-/ALV-U...,"apk ini berguna bagi gw, karna? ya karna buat ...",5,56,3.9.2,2021-08-28 16:33:26,None,NaT,3.9.2
1,6e22e57b-8ed9-4b7c-9003-f1423373e3b5,Yeti Azwar,https://play-lh.googleusercontent.com/a-/ALV-U...,Ini sangat bagus untuk melukis karena bisa men...,5,33,3.9.2,2021-09-26 01:11:47,None,NaT,3.9.2
2,11c06723-35ee-4cf7-8048-5dd85519dcd4,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Selama ini ane pake color picker. Ahirnya nemu...,5,81,3.6.1,2020-02-11 15:02:20,None,NaT,3.6.1
3,e45cd5fa-c658-4bd6-89e0-6d74a53fb27b,Muhammad Khoiruddin,https://play-lh.googleusercontent.com/a-/ALV-U...,Aplikasi ini sangat berguna dalam membaca warn...,4,34,3.9.2,2021-02-15 19:43:29,None,NaT,3.9.2
4,4181e8c9-a1d1-43e5-90c1-9c64c2276431,Fatimia Rahma,https://play-lh.googleusercontent.com/a-/ALV-U...,Aplikasi ini bagus banget untuk mengidentifika...,5,8,3.9.2,2022-10-22 15:33:51,None,NaT,3.9.2


In [21]:
ishihara[['content', 'score']].head()

,content,score
0,"apk ini berguna bagi gw, karna? ya karna buat ...",5
1,Ini sangat bagus untuk melukis karena bisa men...,5
2,Selama ini ane pake color picker. Ahirnya nemu...,5
3,Aplikasi ini sangat berguna dalam membaca warn...,4
4,Aplikasi ini bagus banget untuk mengidentifika...,5


In [22]:
def labelling(score):
    if score < 3:
        return 'Negatif'
    elif score == 3:
        return 'Netral'
    elif score > 3:
        return 'Positif'

ishihara['label'] = ishihara['score'].apply(labelling)
ishihara.head(200)
ishihara = ishihara[['content', 'score']]
ishihara.to_csv('rating4.csv', index=False)

##Color Blindness Test

In [23]:
from google_play_scraper import Sort, reviews

#fungsi untuk memanggil riview
result, continuation_token = reviews(
    'com.hieuit.colorblindtest',
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
    count=1000, #defaultnya 100
)

In [24]:
# Dataframe dengan nama
ishihara = pd.DataFrame(np.array(result),columns=['review'])
ishihara = ishihara.join(pd.DataFrame(ishihara.pop('review').tolist()))
ishihara.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,80e49c29-8cd5-49e4-a5bf-a451c2cbc3ae,Hack Penyadap (Haier),https://play-lh.googleusercontent.com/a/ACg8oc...,"Sangat berguna untuk test buta warna, test pen...",5,2,None,2022-03-27 18:11:38,None,None,None
1,c4f8a4db-b661-4ba3-9731-90291b285b47,JEFFRY ARIANTO,https://play-lh.googleusercontent.com/a/ACg8oc...,mantap nih tapi tambahin lagi jangan sampe 24 aja,4,0,1.4,2022-06-23 23:19:49,None,None,1.4
2,70896840-520b-4c47-a2d3-7991bffa0d56,Bondjol Cool,https://play-lh.googleusercontent.com/a/ACg8oc...,"Bagus, tapi kok sampek 24 saja... tolong dong ...",4,10,1.4,2020-11-04 02:29:35,None,None,1.4
3,e7e60840-e5b5-4e08-a8f9-140469d66142,Iwan Nirwan,https://play-lh.googleusercontent.com/a/ACg8oc...,Boleh lahh,3,0,1.4,2022-06-24 23:12:42,None,None,1.4
4,f133523d-22ec-4bdd-8e10-1ddcfb03f596,Muhammad Fachrul Razi,https://play-lh.googleusercontent.com/a-/ALV-U...,Banyak iklan,1,0,1.4,2022-06-24 09:17:29,None,None,1.4


In [25]:
ishihara[['content', 'score']].head()

,content,score
0,"Sangat berguna untuk test buta warna, test pen...",5
1,mantap nih tapi tambahin lagi jangan sampe 24 aja,4
2,"Bagus, tapi kok sampek 24 saja... tolong dong ...",4
3,Boleh lahh,3
4,Banyak iklan,1


In [26]:
def labelling(score):
    if score < 3:
        return 'Negatif'
    elif score == 3:
        return 'Netral'
    elif score > 3:
        return 'Positif'

ishihara['label'] = ishihara['score'].apply(labelling)
ishihara.head(200)
ishihara = ishihara[['content', 'score']]
ishihara.to_csv('rating5.csv', index=False)

##Color Blind Pal

In [27]:
from google_play_scraper import Sort, reviews

#fungsi untuk memanggil riview
result, continuation_token = reviews(
    'com.softinfosysru.ishiharacolorblindnesstest',
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
    count=1000, #defaultnya 100
)

In [28]:
# Dataframe dengan nama
ishihara = pd.DataFrame(np.array(result),columns=['review'])
ishihara = ishihara.join(pd.DataFrame(ishihara.pop('review').tolist()))
ishihara.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,3c75f655-30c7-47d7-af84-a1ee9375f5c1,Al Vacino,https://play-lh.googleusercontent.com/a-/ALV-U...,Masih belajar untuk dapt melihat lagi Terimaka...,4,9,1.5.2.0,2020-06-21 03:39:57,None,None,1.5.2.0
1,7b3899a4-a7c1-4551-a190-d6434c8ff6cf,Umul Umul,https://play-lh.googleusercontent.com/a-/ALV-U...,"Kalo bisa saran dari saya, untuk test ishihara...",4,6,None,2022-03-04 04:13:29,None,None,None
2,fa08343c-10c2-4672-a41a-b161eed077fc,Alfiyah Alawy,https://play-lh.googleusercontent.com/a/ACg8oc...,Entah kenapa jawaban sering saya dapatkan di p...,4,8,1.5.2.0,2020-08-06 06:57:38,None,None,1.5.2.0
3,93c2180d-4969-4818-ac05-5028e0b92b0e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Kalo bisa ada tingkatan nya juga dong. Dan leb...,5,11,1.5.2.0,2019-12-02 04:29:21,None,None,1.5.2.0
4,5c9c7eef-8066-4d3e-884f-4ff9bfd0cbd4,Mahmud Abbas,https://play-lh.googleusercontent.com/a-/ALV-U...,App ini sangat bermanfaat sekali untuk belajar...,5,2,1.5.2.0,2021-06-14 05:36:22,None,None,1.5.2.0


In [29]:
ishihara[['content', 'score']].head()

,content,score
0,Masih belajar untuk dapt melihat lagi Terimaka...,4
1,"Kalo bisa saran dari saya, untuk test ishihara...",4
2,Entah kenapa jawaban sering saya dapatkan di p...,4
3,Kalo bisa ada tingkatan nya juga dong. Dan leb...,5
4,App ini sangat bermanfaat sekali untuk belajar...,5


In [30]:
def labelling(score):
    if score < 3:
        return 'Negatif'
    elif score == 3:
        return 'Netral'
    elif score > 3:
        return 'Positif'

ishihara['label'] = ishihara['score'].apply(labelling)
ishihara.head(200)
ishihara = ishihara[['content', 'score']]
ishihara.to_csv('rating6.csv', index=False)

##Daltonism Test

In [31]:
from google_play_scraper import Sort, reviews

#fungsi untuk memanggil riview
result, continuation_token = reviews(
    'pl.noip.piekaa.Dalton',
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
    count=1000, #defaultnya 100
)

In [32]:
# Dataframe dengan nama
ishihara = pd.DataFrame(np.array(result),columns=['review'])
ishihara = ishihara.join(pd.DataFrame(ishihara.pop('review').tolist()))
ishihara.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,eae61caa-d646-453d-a67e-747573387651,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sangat bermanfaat untuk mengasah ketajaman mata.,5,9,3,2019-01-01 10:49:17,I'm glad You liked it.\nThanks! ;),2019-01-02 10:33:08,3
1,04554ec0-7daa-4806-9c04-b7554663060d,Budi Sukmajadi,https://play-lh.googleusercontent.com/a-/ALV-U...,Sangat Bagus untuk tes buta warna pada anak da...,5,5,7,2020-12-04 10:44:30,None,NaT,7
2,0188ccc6-1c51-4eca-b8e4-28a4a863df55,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,baik untuk mengetes buta warna pada mata,5,5,3,2019-08-03 04:42:42,None,NaT,3
3,182c848b-e2be-456c-91e4-6c4a2e882bad,Rifqi Syarul Maulana,https://play-lh.googleusercontent.com/a-/ALV-U...,"kata temen gua susah, berguna banget buat teme...",5,4,7,2021-02-15 15:21:06,None,NaT,7
4,4f61dcd9-fe3e-4ea9-808f-eed81c7b1fc7,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Tapi sulit dibuka aplikasinya,5,55,3,2018-10-13 11:17:41,None,NaT,3


In [33]:
ishihara[['content', 'score']].head()

,content,score
0,sangat bermanfaat untuk mengasah ketajaman mata.,5
1,Sangat Bagus untuk tes buta warna pada anak da...,5
2,baik untuk mengetes buta warna pada mata,5
3,"kata temen gua susah, berguna banget buat teme...",5
4,Tapi sulit dibuka aplikasinya,5


In [34]:
def labelling(score):
    if score < 3:
        return 'Negatif'
    elif score == 3:
        return 'Netral'
    elif score > 3:
        return 'Positif'

ishihara['label'] = ishihara['score'].apply(labelling)
ishihara.head(200)
ishihara = ishihara[['content', 'score']]
ishihara.to_csv('rating7.csv', index=False)

##ColorMatch



In [35]:
from google_play_scraper import Sort, reviews

#fungsi untuk memanggil riview
result, continuation_token = reviews(
    'com.rayole.colormatch',
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
    count=1000, #defaultnya 100
)

In [36]:
# Dataframe dengan nama
ishihara = pd.DataFrame(np.array(result),columns=['review'])
ishihara = ishihara.join(pd.DataFrame(ishihara.pop('review').tolist()))
ishihara.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,157a1420-5569-48b6-8881-eab79936fc4f,kes rindam,https://play-lh.googleusercontent.com/a/ACg8oc...,Kenapa punya saya berhenti di hitungan 92? Jad...,3,3,0.1.2,2023-10-20 15:26:55,None,None,0.1.2
1,e7e192e0-34d1-4b0d-aa8b-6a88a3bf00e6,Kheysa Sasa,https://play-lh.googleusercontent.com/a/ACg8oc...,Menurut saya simple dan bagus,5,0,0.1.2,2023-12-22 05:18:27,None,None,0.1.2
2,84698430-cefe-4911-946f-55e7c549e60e,Halili Lili,https://play-lh.googleusercontent.com/a-/ALV-U...,Curang game nya udh bebrpa kali nonton iklan p...,1,0,0.1.2,2023-12-26 18:03:01,None,None,0.1.2
3,41f65228-5318-45c8-ba21-f8c12a7c744a,idiaty amelia,https://play-lh.googleusercontent.com/a-/ALV-U...,Oke,4,0,0.1.2,2023-11-12 14:20:21,None,None,0.1.2
4,36e00625-c4a5-4247-821e-4763ce460a3c,Muhammad Yusuf,https://play-lh.googleusercontent.com/a/ACg8oc...,Ya begitu,4,2,0.1.2,2023-09-20 17:11:56,None,None,0.1.2


In [37]:
def labelling(score):
    if score < 3:
        return 'Negatif'
    elif score == 3:
        return 'Netral'
    elif score > 3:
        return 'Positif'

ishihara['label'] = ishihara['score'].apply(labelling)
ishihara.head(200)
ishihara = ishihara[['content', 'score']]
ishihara.to_csv('rating8.csv', index=False)

# **Menggabungkan file csv**

In [38]:
#menggabungkan file csv
ishihara = ['rating1.csv', 'rating2.csv', 'rating3.csv', 'rating4.csv', 'rating5.csv', 'rating6.csv', 'rating7.csv', 'rating8.csv']

#membuat dataframe kosong utnuk hasil penggabungan file csv
merged_data = pd.DataFrame()

#menggabungkan file csv menjadi 1 dataframe
for file in ishihara:
  df = pd.read_csv(file)
  merged_data = pd.concat([merged_data, df], ignore_index=True) #pd.concat:menggabungkan, ignore_index=True:mereset index DataFrame

#menyimpan hasil penggabungan file csv
merged_data.to_csv('ishihara.csv', index=False)

#menampilkan dataframe hasil penggabungan
print("Jumlah baris dan kolom DataFrame hasil penggabungan:", merged_data.shape)

Jumlah baris dan kolom DataFrame hasil penggabungan: (496, 2)


In [39]:
#mengambil data dari 1 kolom
ishihara = merged_data ['content']
print (ishihara [1:4])

1    ini bagus buat memperbaiki mata.tapi kalo bisa...
2    Saya sih gak buta warna, tapi makin kedepan lv...
3    Coba mengerjakan Ishihara Test sambil mengguna...
Name: content, dtype: object


In [40]:
#melihat semua sel setiap kolm & baris
df = pd.DataFrame(merged_data)
print(df)

                                               content  score
0    Pada tes 'color plates test' kenapa saya selal...      4
1    ini bagus buat memperbaiki mata.tapi kalo bisa...      4
2    Saya sih gak buta warna, tapi makin kedepan lv...      1
3    Coba mengerjakan Ishihara Test sambil mengguna...      4
4                  Tolong aturin pake bahasa Indonesia      3
..                                                 ...    ...
491                                               Good      5
492                                               Good      5
493                                            Goods..      5
494                                          Good luck      5
495                                           Good job      5

[496 rows x 2 columns]


# **Preprocessing**

In [41]:
ishihara = pd.read_csv('/content/drive/MyDrive/Semester5/PemrogramanSistemCerdas1/Prak_IvanderJoshSantoso/Sentimen_analiysis_ishihara/ishihara.csv')
ishihara.head()

,content,score
0,Pada tes 'color plates test' kenapa saya selal...,4
1,ini bagus buat memperbaiki mata.tapi kalo bisa...,4
2,"Saya sih gak buta warna, tapi makin kedepan lv...",1
3,Coba mengerjakan Ishihara Test sambil mengguna...,4
4,Tolong aturin pake bahasa Indonesia,3


##Case Folding

In [42]:
import re
#mengubah huruf besar menjadi kecil

ishihara['content'] = ishihara['content'].str.lower()
ishihara.head()
# # data_casefolding.head()

# data_casefolding = ishihara.applymap(lambda x: x.lower() if isinstance(x, str) else x)
# data_casefolding.head()


# #fungsi untuk case folding
# def casefoldingText(text):
#   #mengubah huruf besar menjadi kecil
#   data_casefolded_text = ishihara.lower()
#   return casefolded_text



,content,score
0,pada tes 'color plates test' kenapa saya selal...,4
1,ini bagus buat memperbaiki mata.tapi kalo bisa...,4
2,"saya sih gak buta warna, tapi makin kedepan lv...",1
3,coba mengerjakan ishihara test sambil mengguna...,4
4,tolong aturin pake bahasa indonesia,3


##Filtering

In [43]:
#kode program menggunakan NLTK untuk melakukan filtering
#filtering
import re

#url
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", ishihara) for ishihara in ishihara['content']]
#cont
filtering_cont = [re.sub(r'\(cont\)'," ", ishihara)for ishihara in filtering_url]
#punctuatuion
filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', ishihara) for ishihara in filtering_cont]
#  hapus #tagger
filtering_tagger = [re.sub(r'#([^\s]+)', '', ishihara) for ishihara in filtering_punctuation]
#numeric
filtering_numeric = [re.sub(r'\d+', ' ', ishihara) for ishihara in filtering_tagger]
#menghapus karakter khusus
filtering_special_characters = [re.sub(r'[^A-Za-z0-9\s]', ' ', ishihara) if "color plates test" not in ishihara else re.sub(r'[^A-Za-z0-9\s]', '', ishihara) for ishihara in filtering_numeric]
data_filtering = pd.Series(filtering_special_characters)
data_filtering.to_csv('data_clean.csv')
print (data_filtering[:9])
#print (data_filtering) #jika ingin mencetak semua data_filtering

0    pada tes color plates test kenapa saya selalu ...
1    ini bagus buat memperbaiki mata tapi kalo bisa...
2    saya sih gak buta warna  tapi makin kedepan lv...
3    coba mengerjakan ishihara test sambil mengguna...
4                  tolong aturin pake bahasa indonesia
5            sangat baik buat test buta warna  lengkap
6               fungsi benar apa tidak hasil tes warna
7    pada awal nya aku udah nyerah karna gagal tes ...
8               bisa liat gak kalo gak berarti katarak
dtype: object


##Tokenization

In [44]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

data_tokens = [word_tokenize(line) for line in data_filtering]
print(data_tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[['pada', 'tes', 'color', 'plates', 'test', 'kenapa', 'saya', 'selalu', 'mendapatkan', 'padahal', 'jawaban', 'saya', 'sudah', 'sesuai', 'dengan', 'jawaban', 'check', 'answer', 'dan', 'sudah', 'saya', 'coba', 'segala', 'jawaban', 'dan', 'tetap', 'mendapatkan', 'hasil'], ['ini', 'bagus', 'buat', 'memperbaiki', 'mata', 'tapi', 'kalo', 'bisa', 'bahasa', 'indonesia', 'aja', 'kalo', 'buat', 'angka', 'masih', 'ngerti', 'kalo', 'bahasa', 'inggrisnya', 'tapi', 'kalo', 'yg', 'lain', 'maaih', 'bisa', 'sedikit', 'ngerti', 'klo', 'bisa', 'di', 'ubah', 'bahasa', 'indonesia', 'aja'], ['saya', 'sih', 'gak', 'buta', 'warna', 'tapi', 'makin', 'kedepan', 'lvlnya', 'gambarnya', 'malah', 'makin', 'gak', 'jelas', 'gak', 'jelasnya', 'bkn', 'soal', 'buta', 'warna', 'tapi', 'makin', 'aneh', 'susah', 'jelasinnya'], ['coba', 'mengerjakan', 'ishihara', 'test', 'sambil', 'menggunakan', 'fitur', 'check', 'answer', 'semuanya', 'normal', 'view', 'tapi', 'skor', 'akhir', 'selalu'], ['tolong', 'aturin', 'pake', 'bahasa

In [45]:
print (data_tokens [:10])

[['pada', 'tes', 'color', 'plates', 'test', 'kenapa', 'saya', 'selalu', 'mendapatkan', 'padahal', 'jawaban', 'saya', 'sudah', 'sesuai', 'dengan', 'jawaban', 'check', 'answer', 'dan', 'sudah', 'saya', 'coba', 'segala', 'jawaban', 'dan', 'tetap', 'mendapatkan', 'hasil'], ['ini', 'bagus', 'buat', 'memperbaiki', 'mata', 'tapi', 'kalo', 'bisa', 'bahasa', 'indonesia', 'aja', 'kalo', 'buat', 'angka', 'masih', 'ngerti', 'kalo', 'bahasa', 'inggrisnya', 'tapi', 'kalo', 'yg', 'lain', 'maaih', 'bisa', 'sedikit', 'ngerti', 'klo', 'bisa', 'di', 'ubah', 'bahasa', 'indonesia', 'aja'], ['saya', 'sih', 'gak', 'buta', 'warna', 'tapi', 'makin', 'kedepan', 'lvlnya', 'gambarnya', 'malah', 'makin', 'gak', 'jelas', 'gak', 'jelasnya', 'bkn', 'soal', 'buta', 'warna', 'tapi', 'makin', 'aneh', 'susah', 'jelasinnya'], ['coba', 'mengerjakan', 'ishihara', 'test', 'sambil', 'menggunakan', 'fitur', 'check', 'answer', 'semuanya', 'normal', 'view', 'tapi', 'skor', 'akhir', 'selalu'], ['tolong', 'aturin', 'pake', 'bahasa

#Modeling

In [46]:
#library yang dibutuhkan untuk modeling dan evaluation
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, accuracy_score, recall_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import joblib

In [47]:
#file ishihara.csv yang sudah di preprocessing
ishihara = pd.read_csv('/content/drive/MyDrive/Semester5/PemrogramanSistemCerdas1/Prak_IvanderJoshSantoso/Sentimen_analiysis_ishihara/data_clean.csv')
ishihara.head()

,Unnamed: 0,0
0,0,pada tes color plates test kenapa saya selalu ...
1,1,ini bagus buat memperbaiki mata tapi kalo bisa...
2,2,saya sih gak buta warna tapi makin kedepan lv...
3,3,coba mengerjakan ishihara test sambil mengguna...
4,4,tolong aturin pake bahasa indonesia


In [48]:
# set label ke dalam variabel y
y = df['score']
y[0:10]  # baris ke 0 dan kolom ke 10

0    4
1    4
2    1
3    4
4    3
5    5
6    2
7    5
8    4
9    5
Name: score, dtype: int64

In [49]:
reviews=df.content
reviews

0      Pada tes 'color plates test' kenapa saya selal...
1      ini bagus buat memperbaiki mata.tapi kalo bisa...
2      Saya sih gak buta warna, tapi makin kedepan lv...
3      Coba mengerjakan Ishihara Test sambil mengguna...
4                    Tolong aturin pake bahasa Indonesia
                             ...                        
491                                                 Good
492                                                 Good
493                                              Goods..
494                                            Good luck
495                                             Good job
Name: content, Length: 496, dtype: object

In [50]:
vectorizer = TfidfVectorizer()
reviews2=["". join(r) for r in reviews]
reviews2

["Pada tes 'color plates test' kenapa saya selalu mendapatkan 1/2? Padahal jawaban saya sudah sesuai dengan jawaban 'Check answer' dan sudah saya coba segala jawaban dan tetap mendapatkan hasil 1/2",
 'ini bagus buat memperbaiki mata.tapi kalo bisa bahasa indonesia aja.kalo buat angka masih ngerti kalo bahasa inggrisnya.tapi kalo yg lain maaih bisa sedikit ngerti.klo bisa di ubah bahasa indonesia aja',
 'Saya sih gak buta warna, tapi makin kedepan lvlnya gambarnya malah makin gak jelas, gak jelasnya bkn soal buta warna tapi makin aneh, susah jelasinnya',
 'Coba mengerjakan Ishihara Test sambil menggunakan fitur "check answer". Semuanya "normal view" tapi skor akhir selalu 36/37',
 'Tolong aturin pake bahasa Indonesia',
 'Sangat baik buat test buta warna ,lengkap',
 'Fungsi benar apa tidak hasil tes warna',
 'Pada awal nya aku udah nyerah karna gagal tes buta warna padahal aku bisa tau warna apa saja yang aku lihat,tapi setelah di liat liat ternyata aku bisa,benar kata ibuku"KAMU GAK BU

In [53]:
 X = vectorizer.fit_transform(reviews2)
 y = df['score']

In [54]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80)

In [55]:
import pickle

with open('review.pkl','wb') as file:
  pickle.dump(X,file)


In [56]:

for c in [0.01, 0.05, 0.25, 0.5, 1]:
  lr = LogisticRegression(C=c)
  lr.fit(X, y)
  print('Accuracy for C=%s: %s'
  % (c, accuracy_score(y_test, lr.predict(X_test))))

Accuracy for C=0.01: 0.71
Accuracy for C=0.05: 0.71
Accuracy for C=0.25: 0.71
Accuracy for C=0.5: 0.72
Accuracy for C=1: 0.78


In [57]:
#find the best value of c in support vector
for c in [0.01, 0.05, 0.25, 0.5, 1]:
  sv =  SVC(C=c)
  sv.fit(X, y)
  print('Accuracy for C=%s: %s'
  % (c, accuracy_score(y_test, sv.predict(X_test))))

Accuracy for C=0.01: 0.71
Accuracy for C=0.05: 0.71
Accuracy for C=0.25: 0.71
Accuracy for C=0.5: 0.75
Accuracy for C=1: 0.92


In [58]:
#here I choose C=0.5 to build the final model for Logistic Regression
final_model_lr = LogisticRegression(C=0.5)
final_model_lr.fit(X, y)
print('final model accuracy: %s' %accuracy_score(y_test, final_model_lr.predict(X_test)))

final model accuracy: 0.72


In [59]:
#here I choose C=0.25 to build the final model for Support Vector (SVM)
final_model_sv = SVC(C=0.25)
final_model_sv.fit(X, y)
print('final model accuracy: %s' %accuracy_score(y_test, final_model_lr.predict(X_test)))

final model accuracy: 0.72


In [60]:


import pickle

In [61]:
with open('model_svm.pkl','wb') as file:
  pickle.dump(final_model_sv,file)

In [62]:
# Simpan model tfidf_vectorizer
with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

#Evaluasi model K-NN